### Travelling Salesman Problem Using Genetic Algorithm 

The Travelling Salesman Problem (TSP) is an optimisation problem where a salesman needs to visit all cities from a given list once to reach their starting point with the shortest possible route.

We will use a genetic algorithm to solve this. A genetic algorithm requires these parameters, 

- Number of generations
- Population size (Cities and their distance information)
- Crossover rate
- Mutation rate
- Selection method (fitness method) - we will use roulette wheel selection

In [22]:
import pandas as pd
import numpy as np
import random as rand

class GeneticAlgorithm:
    def __init__(self, population, generations, cities, distance_map, crossover_rate, mutation_rate):
        self.population = population
        self.generations = generations
        self.cities = cities
        self.distance_map = distance_map
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate

    def initialise(self):
        pop_bag = []
        pop_bag.append(self.cities)
        for i in range(self.population):
            curr = self.cities.copy()
            rand.shuffle(curr)
            pop_bag.append(curr)
        return np.array(pop_bag)
        
    def calculate_distance(self, individual):
        total_distance = 0.0
        for i in range(len(individual) - 1):
           total_distance += self.distance_map[self.cities.index(individual[i]), self.cities.index(individual[i + 1])]
        return total_distance

    ### Fitness is inverse of distance
    def fitness(self, population):
        fitness_score = []
        for individual in population:
            distance = self.calculate_distance(individual)
            fitness = 1 / (distance + 1e-6)
            fitness_score.append(fitness)
        return np.array(fitness_score)
    
    def roulette_wheel_selection(self, population, fitness):
        total_fitness = np.sum(fitness)
        probabilities = fitness / total_fitness
        cumulative_prob = np.cumsum(probabilities)
        random_value = rand.random()
        for index, prob in enumerate(cumulative_prob):
            if random_value >= prob:
                return population[index]
        return population[0]

    def mutation(self, individual):
        if rand.random() <= self.mutation_rate:
            size = len(individual)
            first, second = sorted(rand.sample(range(size), 2))
            individual[first], individual[second] = individual[second], individual[first]
        return individual
    
    def crossover(self, parent1, parent2):
        if rand.random() > self.crossover_rate:
            return parent1.copy()
        size = len(parent1)
        start, end = sorted(rand.sample(range(size), 2))
        child = [-1] * size
        child[start:end+1] = parent1[start:end+1] ## pick rest from parent2
        ptr = 0
        for chromosome in parent2:
            if chromosome not in child:
                while child[ptr] != -1:
                    ptr += 1
                child[ptr] = chromosome    
        return child

    def next_generation(self, population):
        fitness = self.fitness(population)
        new_population = []
        
        for _ in range(self.population):
            parent1 = self.roulette_wheel_selection(population, fitness)
            parent2 = self.roulette_wheel_selection(population, fitness)

            child = self.crossover(parent1, parent1)
            child = self.mutation(child)
            new_population.append(child)

        return np.array(new_population)

    def run(self):
        population = self.initialise()
        for gen in range(self.generations):
            population = self.next_generation(population)
            best_path = min(population, key=self.calculate_distance)
        return best_path

if __name__ == '__main__':
    cities = [0, 1, 2, 3, 4, 5]
    distance_map = np.array([
        [0.00, 38.02, 27.12, 33.46, 64.07, 42.1],
        [38.02, 0.00, 31.00, 29.55, 35.55, 38.4],
        [27.12, 31.00, 0.00, 28.03, 47.38, 57.9],
        [33.46, 29.55, 28.03, 0.00, 55.11, 20.07],
        [64.07, 35.55, 47.38, 55.11, 0.00, 16.02],
        [42.1, 38.4, 57.9, 20.07, 16.02, 0.00]
    ])
    
    ga = GeneticAlgorithm(
        population=20,
        generations=30,
        cities=cities,
        distance_map=distance_map,
        crossover_rate=0.9,
        mutation_rate=0.1
    )

    best = ga.run()
    print("Best Path Found:", best)
    print("Best Distance:", ga.calculate_distance(best))

Best Path Found: [0 3 1 2 4 5]
Best Distance: 157.41000000000003
